# Sentence Embeddings

This notebook contains the code to generate sentence embeddings using the pre-trained models from the [sentence-transformers](https://www.sbert.net/index.html) library.

In [1]:
import pandas as pd
from pathlib import Path

from sentence_transformers import SentenceTransformer

In [2]:
PATH_DATA_BASE = Path.cwd().parent / "data"
PATH_SENTENCES = Path.cwd().parent / "models/sentences"
PATH_EMBEDDINGS = Path.cwd().parent / "models/embeddings"

In [3]:
# Setting pandas option to display the full content of DataFrame columns without truncation
pd.set_option('display.max_colwidth', None)

In [4]:
dataset = pd.read_csv(PATH_DATA_BASE / 'filtered_data.csv')
dataset.head()

,titles,abstracts,terms,urls
0,Compositionally Equivariant Representation Learning,"Deep learning models often need sufficient supervision (i.e. labelled data)\nin order to be trained effectively. By contrast, humans can swiftly learn to\nidentify important anatomy in medical images like MRI and CT scans, with\nminimal guidance. This recognition capability easily generalises to new images\nfrom different medical facilities and to new tasks in different settings. This\nrapid and generalisable learning ability is largely due to the compositional\nstructure of image patterns in the human brain, which are not well represented\nin current medical models. In this paper, we study the utilisation of\ncompositionality in learning more interpretable and generalisable\nrepresentations for medical image segmentation. Overall, we propose that the\nunderlying generative factors that are used to generate the medical images\nsatisfy compositional equivariance property, where each factor is compositional\n(e.g. corresponds to the structures in human anatomy) and also equivariant to\nthe task. Hence, a good representation that approximates well the ground truth\nfactor has to be compositionally equivariant. By modelling the compositional\nrepresentations with learnable von-Mises-Fisher (vMF) kernels, we explore how\ndifferent design and learning biases can be used to enforce the representations\nto be more compositionally equivariant under un-, weakly-, and semi-supervised\nsettings. Extensive results show that our methods achieve the best performance\nover several strong baselines on the task of semi-supervised domain-generalised\nmedical image segmentation. Code will be made publicly available upon\nacceptance at https://github.com/vios-s.","['cs.CV', 'cs.LG']",http://arxiv.org/abs/2306.07783v1
1,Aria Digital Twin: A New Benchmark Dataset for Egocentric 3D Machine Perception,"We introduce the Aria Digital Twin (ADT) - an egocentric dataset captured\nusing Aria glasses with extensive object, environment, and human level ground\ntruth. This ADT release contains 200 sequences of real-world activities\nconducted by Aria wearers in two real indoor scenes with 398 object instances\n(324 stationary and 74 dynamic). Each sequence consists of: a) raw data of two\nmonochrome camera streams, one RGB camera stream, two IMU streams; b) complete\nsensor calibration; c) ground truth data including continuous\n6-degree-of-freedom (6DoF) poses of the Aria devices, object 6DoF poses, 3D eye\ngaze vectors, 3D human poses, 2D image segmentations, image depth maps; and d)\nphoto-realistic synthetic renderings. To the best of our knowledge, there is no\nexisting egocentric dataset with a level of accuracy, photo-realism and\ncomprehensiveness comparable to ADT. By contributing ADT to the research\ncommunity, our mission is to set a new standard for evaluation in the\negocentric machine perception domain, which includes very challenging research\nproblems such as 3D object detection and tracking, scene reconstruction and\nunderstanding, sim-to-real learning, human pose prediction - while also\ninspiring new machine perception tasks for augmented reality (AR) applications.\nTo kick start exploration of the ADT research use cases, we evaluated several\nexisting state-of-the-art methods for object detection, segmentation and image\ntranslation tasks that demonstrate the usefulness of ADT as a benchmarking\ndataset.","['cs.CV', 'cs.AI', 'cs.LG']",http://arxiv.org/abs/2306.06362v2
2,Mean Shift Mask Transformer for Unseen Object Instance Segmentation,"Segmenting unseen objects from images is a critical perception skill that a\nrobot needs to acquire. In robot manipulation, it can facilitate a robot to\ngrasp and manipulate unseen objects. Mean shift clustering is a widely used\nmethod for image segmentation tasks. However, the traditional mean shift\nclustering algorithm is not differentiable, making it difficult to integrate it\ninto an end-to-end neural network trainin

## sentence-transformers models

### What is a sentence-transformers model?

It maps sentences & paragraphs to a N dimensional dense vector space and can be used for tasks like clustering or semantic search.

### all-MiniLM-L6-v2

MiniLM is a smaller variant of the BERT model which has been designed to provide high-quality language understanding capabilities while being significantly smaller and more efficient. The "`all-MiniLM-L6-v2`" model refers to a specific configuration of the MiniLM model.

Here are some reasons why I have chosen this model for my project:

- Efficiency: MiniLM models are smaller and faster than full-size BERT models, which can be a major advantage if you're working on a project with limited computational resources or if you need to process large amounts of data quickly.
- Performance: Despite their smaller size, MiniLM models often perform at a comparable level to full-size BERT models on a variety of NLP tasks. This means that you can often use a MiniLM model without sacrificing much in the way of performance. In fact, the `Performance Sentence Embeddings` metric which is the average performance on encoding sentences over 14 diverse tasks from different domains is `68.06` for the `all-MiniLM-L6-v2` model, which is very good to start with.
- Ease of Use: If you're using a library like Hugging Face's Transformers, it can be relatively straightforward to load a pre-trained MiniLM model and fine-tune it for your specific task.
- Lower Memory Requirements: Given its smaller size, MiniLM requires less memory for training and inference. This could be a crucial factor if you're working with limited hardware resources.

In [5]:
model = SentenceTransformer('all-MiniLM-L6-v2')

# Our feature we like to encode
sentences = dataset['titles']

# Features are encoded by calling model.encode()
embeddings = model.encode(sentences)

In [12]:
# Print the embeddings
c = 0
for sentence, embedding in zip(sentences, embeddings):

    print("Sentence:", sentence)
    print("Embedding dimension:", len(embedding))
    print("Title length:", len(sentence))
    print("")

    if c >=5:
        break
    c +=1 

Sentence: Compositionally Equivariant Representation Learning
Embedding dimension: 384
Title length: 51

Sentence: Aria Digital Twin: A New Benchmark Dataset for Egocentric 3D Machine Perception
Embedding dimension: 384
Title length: 79

Sentence: Mean Shift Mask Transformer for Unseen Object Instance Segmentation
Embedding dimension: 384
Title length: 67

Sentence: AerialFormer: Multi-resolution Transformer for Aerial Image Segmentation
Embedding dimension: 384
Title length: 72

Sentence: VPUFormer: Visual Prompt Unified Transformer for Interactive Image Segmentation
Embedding dimension: 384
Title length: 79

Sentence: AutoSAM: Adapting SAM to Medical Images by Overloading the Prompt Encoder
Embedding dimension: 384
Title length: 73



In [7]:
import pickle

# Saving sentences and corresponding embeddings
with open(PATH_EMBEDDINGS / 'embeddings.pkl', 'wb') as f:
    pickle.dump(embeddings, f)

with open(PATH_SENTENCES / 'sentences.pkl', 'wb') as f:
    pickle.dump(sentences, f)

## Testing the embedding model

In [13]:
paper_you_like = input("Enter your topic of interest here 👇 \n")
paper_you_like

'Language models'

In [14]:
from sentence_transformers import util
cosine_scores = util.cos_sim(embeddings, model.encode(paper_you_like))

In [15]:
import torch
top_similar_papers = torch.topk(cosine_scores,dim=0, k=5,sorted=True)
top_similar_papers

torch.return_types.topk(
values=tensor([[0.7386],
        [0.7290],
        [0.7224],
        [0.7080],
        [0.7038]]),
indices=tensor([[42238],
        [42288],
        [42625],
        [42476],
        [42319]]))

In [16]:
for i in top_similar_papers.indices:
#     print(i)
    print(sentences[i.item()])

Large Language Models as Tool Makers
Large Language Model Programs
Knowledge is a Region in Weight Space for Fine-tuned Language Models
Fine-Tuning Language Models with Just Forward Passes
On the Creativity of Large Language Models
